In [92]:
import re
import numpy as np
import pandas as pd
import string
# plotting
import seaborn as sns
import matplotlib.pyplot as plt
# nltk
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from collections import defaultdict
#Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action = 'ignore')
 
import gensim
from gensim.models import Word2Vec

# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package punkt to C:\Users\BALAJI
[nltk_data]     RAMYA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\BALAJI
[nltk_data]     RAMYA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\BALAJI
[nltk_data]     RAMYA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Loading the Dataset

In [70]:
DATASET_COLUMNS = ['claims_id','claims','status']
DATASET_ENCODING = "ISO-8859-1"
DATASET_SEP=","
DATASET_FILENAME = 'C:\\ucrise_dataset_project_final\\u_crise_dataset.csv'
df = pd.read_csv(DATASET_FILENAME, sep=DATASET_SEP, encoding=DATASET_ENCODING, names=DATASET_COLUMNS)

In [71]:
df

,claims_id,claims,status
0,claims_id,claims,Status
1,1,Putin thinks hell break Americas will in Ukr...,C
2,2,Russia says it will ramp up operations as rock...,C
3,3,Russia threatens broad Ukraine offensive as U....,C
4,4,UN to investigate the killing and wounding of ...,C
...,...,...,...
263,263,Russian forces shift target to Donetsk as Zele...,F
264,264,I don't think he would. If he going to shoot c...,F
265,265,Only way to end it...is to expand it first. ð...,F
266,266,CNBC: Russia says Biden's 'war criminal' claim...,F


In [72]:
data=df[['claims','status']]

In [73]:
data['claims'].unique()

array(['claims',
       'Putin thinks he\x92ll break America\x92s will in Ukraine, but he\x92s wrong, says the CIA director',
       'Russia says it will ramp up operations as rockets pound Ukrainian cities',
       'Russia threatens broad Ukraine offensive as U.S. presses China over war stance',
       'UN to investigate the killing and wounding of children in Ukraine',
       'The West\x92s unity over Ukraine could be starting to crack, just as Russia\x92s invasion gains ground',
       'G-20 finance meetings in Bali overshadowed by war in Ukraine',
       '\x91Frozen conflict\x92: Ukraine war could last 10 years or more if Russian forces aren\x92t pushed back',
       'Russia\x92s squeeze on gas means Germany\x92s energy giant is having to draw supplies from storage',
       'U.S. calls on Russia to halt forced deportations of Ukrainians, citing war crimes',
       'Ukraine Muslims pray for victory, end of occupation',
       'Russia\x92s Ukraine offensive now relying on a private m

In [74]:
print(f"Total Count = {df.count()}")

Total Count = claims_id    268
claims       268
status       268
dtype: int64


In [75]:
#Lowercase Transformation
dataset = df['claims'].str.lower()

In [76]:
df

,claims_id,claims,status
0,claims_id,claims,Status
1,1,Putin thinks hell break Americas will in Ukr...,C
2,2,Russia says it will ramp up operations as rock...,C
3,3,Russia threatens broad Ukraine offensive as U....,C
4,4,UN to investigate the killing and wounding of ...,C
...,...,...,...
263,263,Russian forces shift target to Donetsk as Zele...,F
264,264,I don't think he would. If he going to shoot c...,F
265,265,Only way to end it...is to expand it first. ð...,F
266,266,CNBC: Russia says Biden's 'war criminal' claim...,F


In [77]:
#Stopwords Removal
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [78]:
stop_words = set(stopwordlist)
def remove_stop_words(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
df['claims'] = df['claims'].apply(lambda text: remove_stop_words(text))
df_stopwords = df['claims']
print(f"Stop Words : {df_stopwords}")

Stop Words : 0                                                 claims
1      Putin thinks hell break Americas Ukraine, bu...
2      Russia says ramp operations rockets pound Ukra...
3      Russia threatens broad Ukraine offensive U.S. ...
4       UN investigate killing wounding children Ukraine
                             ...                        
263    Russian forces shift target Donetsk Zelenskyy ...
264    I don't think would. If going shoot cruise mis...
265    Only way end it...is expand first. ð½ðºð...
266    CNBC: Russia says Biden's 'war criminal' claim...
267    Russia's messages missiles tell West back off....
Name: claims, Length: 268, dtype: object


In [79]:
#Punctuations Removal
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
df['claims']= df['claims'].apply(lambda x: remove_punctuations(x))
print(df['claims'])

0                                                 claims
1      Putin thinks hell break Americas Ukraine but...
2      Russia says ramp operations rockets pound Ukra...
3      Russia threatens broad Ukraine offensive US pr...
4       UN investigate killing wounding children Ukraine
                             ...                        
263    Russian forces shift target Donetsk Zelenskyy ...
264    I dont think would If going shoot cruise missi...
265    Only way end itis expand first ð½ðºð¦ US...
266    CNBC Russia says Bidens war criminal claim put...
267    Russias messages missiles tell West back off T...
Name: claims, Length: 268, dtype: object


In [80]:
#Repeating Characters Removal
def remove_repeating_character(text):
    return re.sub(r'(.)1+', r'1', text)
df['claims'] = df['claims'].apply(lambda x: remove_repeating_character(x))
print(df['claims'].head())


0                                               claims
1    Putin thinks hell break Americas Ukraine but...
2    Russia says ramp operations rockets pound Ukra...
3    Russia threatens broad Ukraine offensive US pr...
4     UN investigate killing wounding children Ukraine
Name: claims, dtype: object


In [81]:
#URL Removal
def remove_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
df['claims'] = df['claims'].apply(lambda x: remove_URLs(x))
print(df['claims'].head())

0                                               claims
1    Putin thinks hell break Americas Ukraine but...
2    Russia says ramp operations rockets pound Ukra...
3    Russia threatens broad Ukraine offensive US pr...
4     UN investigate killing wounding children Ukraine
Name: claims, dtype: object


In [82]:
#Numbers Removal
def remove_numbers(data):
    return re.sub('[0-9]+', '', data)
df['claims'] = df['claims'].apply(lambda x: remove_numbers(x))
df['claims'].head()

0                                               claims
1    Putin thinks hell break Americas Ukraine but...
2    Russia says ramp operations rockets pound Ukra...
3    Russia threatens broad Ukraine offensive US pr...
4     UN investigate killing wounding children Ukraine
Name: claims, dtype: object

In [83]:
#Word Tokenization
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'w+')
df['claims'] = df['claims'].apply(tokenizer.tokenize)
df['claims'].head()

0     []
1    [w]
2     []
3    [w]
4    [w]
Name: claims, dtype: object

In [84]:
#Stemming
st = nltk.PorterStemmer()
def stem_text(data):
    text = [st.stem(word) for word in data]
    return data
df['claims'] = df['claims'].apply(lambda x: stem_text(x))
df['claims'] = df['claims'].astype('string')

df_stemming = df['claims']

print(df_stemming)

0                        []
1                     ['w']
2                        []
3                     ['w']
4                     ['w']
               ...         
263         ['w', 'w', 'w']
264         ['w', 'w', 'w']
265         ['w', 'w', 'w']
266                   ['w']
267    ['w', 'w', 'w', 'w']
Name: claims, Length: 268, dtype: string


In [85]:
#TF_IDF

X=data.claims
y=data.status

In [96]:
# Separating the 95% data for training data and 5% for testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state =26105111)

vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
vectoriser.fit(X_train)
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

print(f"X_Train = {X_train}")
print(f"X_test = {X_test}")

No. of feature_words:  3850
X_Train =   (0, 3782)	0.2190037650392608
  (0, 3781)	0.2190037650392608
  (0, 3592)	0.2190037650392608
  (0, 3586)	0.16010842081916177
  (0, 3397)	0.1628502760300441
  (0, 3374)	0.053663630556688866
  (0, 2659)	0.18007605994795534
  (0, 2654)	0.05896661495179495
  (0, 2341)	0.24845143714931034
  (0, 2273)	0.15510452509338885
  (0, 2257)	0.1025618886322205
  (0, 2204)	0.2190037650392608
  (0, 2198)	0.09391218847611608
  (0, 1574)	0.2190037650392608
  (0, 1571)	0.14657922915654129
  (0, 1556)	0.06474493989588642
  (0, 1555)	0.06474493989588642
  (0, 1491)	0.2190037650392608
  (0, 1490)	0.2190037650392608
  (0, 1355)	0.2190037650392608
  (0, 1354)	0.16578136990802436
  (0, 727)	0.24845143714931034
  (0, 630)	0.06474493989588642
  (0, 569)	0.2190037650392608
  (0, 565)	0.15062838783790578
  :	:
  (253, 2297)	0.16522402327017285
  (253, 2290)	0.1102471582926828
  (253, 2274)	0.1456408688405236
  (253, 2257)	0.06820523184906807
  (253, 2158)	0.16522402327017285
  

In [93]:
DATASET_ENCODING = "ISO-8859-1"
#  Reads ‘alice.txt’ file
sample = open('C:\\ucrise_dataset_project_final\\u_crise_dataset.csv',encoding=DATASET_ENCODING)
s = sample.read()
# Replaces escape character with space
f = s.replace("\n", " ")
data = []
# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [94]:
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)
# Print results
print("Cosine similarity between 'ukraine' " + 
               "and 'war' - CBOW : ",
    model1.wv.similarity('ukraine', 'war'))
     
print("Cosine similarity between 'putin' " +
                 "and 'zelenskyy' - CBOW : ",
      model1.wv.similarity('putin', 'zelenskyy'))
 
print("Cosine similarity between 'ukrainewar' " +
                 "and 'russia' - CBOW : ",
      model1.wv.similarity('ukrainewar', 'russia'))

Cosine similarity between 'ukraine' and 'war' - CBOW :  0.99721706
Cosine similarity between 'putin' and 'zelenskyy' - CBOW :  0.9894582
Cosine similarity between 'ukrainewar' and 'russia' - CBOW :  0.6306211


In [95]:
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100,
                                             window = 5, sg = 1)
 
# Print results
print("Cosine similarity between 'ukraine' " +
          "and 'war' - Skip Gram : ",
    model2.wv.similarity('ukraine', 'war'))
     
print("Cosine similarity between 'putin' " +
            "and 'zelenskyy' - Skip Gram : ",
      model2.wv.similarity('putin', 'zelenskyy'))

print("Cosine similarity between 'ukrainewar' " +
                 "and 'russia' - Skip Gram  : ",
      model2.wv.similarity('ukrainewar', 'russia'))

Cosine similarity between 'ukraine' and 'war' - Skip Gram :  0.99774635
Cosine similarity between 'putin' and 'zelenskyy' - Skip Gram :  0.9978709
Cosine similarity between 'ukrainewar' and 'russia' - Skip Gram  :  0.986295


In [87]:
#Evaluation of Models
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)
    categories = ['Claims','True','False']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1}n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

In [88]:
'#BernoulliNB Model'
print("############# Bernoulli Model #####################")
BNBmodel = BernoulliNB()
BNBmodel.fit(X_train, y_train)
model_Evaluate(BNBmodel)
y_pred1 = BNBmodel.predict(X_test)

############# Bernoulli Model #####################
              precision    recall  f1-score   support

           C       0.55      0.86      0.67         7
           F       0.00      0.00      0.00         3
           T       0.00      0.00      0.00         4

    accuracy                           0.43        14
   macro avg       0.18      0.29      0.22        14
weighted avg       0.27      0.43      0.33        14



In [89]:
'#Linear SVC model'
print("############# Linear SVC Model #####################")
SVCmodel = LinearSVC()
SVCmodel.fit(X_train, y_train)
model_Evaluate(SVCmodel)
y_pred2 = SVCmodel.predict(X_test)

############# Linear SVC Model #####################
              precision    recall  f1-score   support

           C       1.00      1.00      1.00         7
           F       1.00      0.33      0.50         3
           T       0.67      1.00      0.80         4

    accuracy                           0.86        14
   macro avg       0.89      0.78      0.77        14
weighted avg       0.90      0.86      0.84        14



In [90]:
print("############# Logistic Model #####################")
LRmodel = LogisticRegression(C=2, max_iter=1000, n_jobs=-1)
LRmodel.fit(X_train, y_train)
model_Evaluate(LRmodel)
y_pred3 = LRmodel.predict(X_test)

############# Logistic Model #####################
              precision    recall  f1-score   support

           C       0.17      0.14      0.15         7
           F       0.00      0.00      0.00         3
           T       0.00      0.00      0.00         4

    accuracy                           0.07        14
   macro avg       0.06      0.05      0.05        14
weighted avg       0.08      0.07      0.08        14



In [91]:
print("############# Random Forest Model #####################")
RFmodel = RandomForestClassifier()
RFmodel.fit(X_train, y_train)
model_Evaluate(RFmodel)
y_pred3 = RFmodel.predict(X_test)

############# Random Forest Model #####################
              precision    recall  f1-score   support

           C       0.88      1.00      0.93         7
           F       0.00      0.00      0.00         3
           T       0.67      1.00      0.80         4

    accuracy                           0.79        14
   macro avg       0.51      0.67      0.58        14
weighted avg       0.63      0.79      0.70        14

